# <div align="center"><b>Differential expression analysis with DESeq2</b></div>

We will use the DESeq2 package to perform differential expression analysis our counts data.

In [ ]:
# Start by importing useful packages
import numpy as np
import pandas as pd
import seaborn as sns

# DESeq2 imports
from pydeseq2.dds import DeseqDataSet  # this is for the data structure we will create for DESeq2
from pydeseq2.default_inference import DefaultInference  # this is for the inference engine we will use
from pydeseq2.ds import DeseqStats  # 

# 
import decoupler as dc  # Helps us a plot a volcano plot :)

In [ ]:
# It can also be useful to specify all your paths here so it is clear where things are coming from
# TODO: Make sure this matches the path of your counts file
path_counts = "~/scratch/feature_counts/hangauer.results.counts"# 
path_out = '~/scratch/differential_analysis'

In [ ]:
# We can actually use python to create a directory if it doesn't exist
import os

if not os.path.exists(path_out):
    os.makedirs(path_out)

# 1) Load data

We will load our data the same way we did in the previous notebook.

In [ ]:
# Load counts_df
counts_df = pd.read_csv(path_counts, comment="#", sep="\t", index_col=0)
gene_info = counts_df.iloc[:, 0:5]
counts_df = counts_df.iloc[:, 5:]
counts_df = counts_df.rename(
    columns=
    {
        counts_df.columns[0]:"Parental1",
        counts_df.columns[1]:"Parental2",
        counts_df.columns[2]:"Persister1",
        counts_df.columns[3]:"Persister2",
        counts_df.columns[4]:"Persister3"
    }
)
counts_df = counts_df.T
counts_df.head()

In [ ]:
# We now need to create a sample information table with the condition of each sample
metadata = pd.DataFrame(index=counts_df.index)
metadata["condition"] = metadata.index.str[:-1]
metadata

In [ ]:
# Filtering out any samples that don't have a condition
samples_to_keep = ~metadata.condition.isna().values
counts_df = counts_df.loc[samples_to_keep]
metadata = metadata.loc[samples_to_keep]

In [ ]:
# Filter genes that have less than 10 counts in all samples
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

# 2) Running DESeq2

In [ ]:
# This let's us use multiple processors (cpus) to speed up the analysis
inference = DefaultInference(n_cpus=4)

In [ ]:
# This is the data structure that DESeq2 will use
# It is common for packages to implement their own data structures 
# that store all the information needed for the analysis
dds = DeseqDataSet(
    counts=counts_df,  # The counts table
    metadata=metadata,  # The sample information table
    design_factors="condition",  # The column in the metadata that specifies the condition with which to compare
    refit_cooks=True,  # This is a setting that can help with outliers
    inference=inference,  # The inference engine to use
)

In [ ]:
# Run the DESeq2 pipeline!!! It's that easy!
dds.deseq2()

In [ ]:
# Extract the results of the analysis for the comparison of Persister vs Parental
stat_res = DeseqStats(dds, inference=inference, contrast=["condition", "Persister", "Parental"])
stat_res.summary()

In [ ]:
# Grab the dataframe with the differential expression results and print the first few rows
res = stat_res.results_df
res.head()

# 3) Mapping to gene symbols

The featureCounts tool we used to count reads used a different identifier for genes than the symbols most people are familiar with. So we need to map the gene identifiers to gene symbols. We use a function from the sanbomics package to do this.

In [ ]:
# Let's import the specific function we need from sanbomics
from sanbomics.tools import id_map

In [ ]:
# Let's load the id mapper for human
mapper = id_map(species = 'human')
mapper

In [ ]:
# We can now map the ensembl gene ids to gene symbols and check it
res["ensembl_gene_id"] = res.index.str.split(".").str[0]
res['Symbol'] = res["ensembl_gene_id"].map(mapper.mapper) 
res.head()

In [ ]:
# Let's filter out genes that have a baseMean less than 10
res = res[res.baseMean >= 10]
res

In [ ]:
# Finally, we can save the results to a csv file using pandas
res.to_csv(f"{path_out}/deseq2_results.csv")

# 4) Some useful visualizations

## Volcano plot

Volcano plots are a useful way to visualize the results of a differential expression analysis. The x-axis shows the log2 fold change and the y-axis shows the -log10 of the p-value. The points are colored based on whether they are significantly differentially expressed or not.

In [ ]:
# Let's do a little bit more manipulation to make the results so we see gene symbols when we plot
res["Geneid"] = res.index
res = res.set_index("Symbol")

In [ ]:
# Let's use the decoupler package to plot a volcano plot
dc.plot_volcano_df(
    res,  # The results table
    x='log2FoldChange',  # The column with the log2 fold changes will be on the x-axis
    y='padj',  # The column with the adjusted p-values will be on the y-axis
    top=20,  # The number of top genes to label
    figsize=(5, 5),  # The size of the figure
    sign_thr=0.05,  # The significance threshold to use for padj
    lFCs_thr=1,  # The log2 fold change threshold to use
)

## Heatmap

Heatmaps are another useful way to visualize the expression of genes across samples. They show the genes the significantly differ between samples and can be used to cluster samples based on their expression profiles.

In [ ]:
# Let's filter down to the significant genes
sigs = res[(res.padj < 0.001) & (abs(res.log2FoldChange) > 2)]
sigs

In [ ]:
# Let's manipulate the dds object to only include the significant genes and store log1p transformed counts
dds.layers['log1p'] = np.log1p(dds.layers['normed_counts'])
dds_sigs = dds[:, sigs.index]
dds_sigs

In [ ]:
# Let's create a dataframe with the log1p transformed counts
grapher = pd.DataFrame(
    dds_sigs.layers['log1p'].T,
    index=dds_sigs.var_names, 
    columns=dds_sigs.obs_names
)

In [ ]:
# Finally, let's plot the heatmap (with clustering!)
sns.clustermap(grapher, z_score=0, cmap = 'RdYlBu_r')

## Principal component analysis (PCA)

PCA is a useful way to visualize the variation in the data. It can be used to see if the samples cluster based on their expression profiles. We will use the ScanPy package to perform PCA. ScanPy is a powerful package for single-cell RNA-seq analysis, but it can also be used for bulk RNA-seq analysis.

In [ ]:
import scanpy as sc  # load the scanpy package

In [ ]:
# Run PCA on the dds object and plot the results
sc.tl.pca(dds)
sc.pl.pca(dds, color = 'condition', size = 200)

# 5) Some potential exercises

1. How many genes are significantly differentially expressed between the two groups? How many are upregulated and how many are downregulated?
2. What are the top 10 most upregulated genes? What are the top 10 most downregulated genes? Plot these genes on a heatmap.
3. What are the top 10 most variable genes across samples? Do these match the top 10 most differentially expressed genes?

# DONE!

---